In [23]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import preprocessing
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from keras.layers import Convolution2D ,Activation, Conv2D,MaxPooling2D,Flatten,Dense,Dropout


In [24]:
train_dir = r'C:\Users\mayan\Downloads\New Datasets\skin disease kushal\new\train'
test_dir = r'C:\Users\mayan\Downloads\New Datasets\skin disease kushal\new\test'

In [25]:
train_datagen = ImageDataGenerator(rescale=1./255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True,
                          rotation_range=0.2,
                          width_shift_range=0.0,
                          height_shift_range=0.0)

test_datagen = ImageDataGenerator(rescale=1./255)


In [26]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                         target_size=(64,64),
                                         batch_size = 32,
                                         class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                         target_size=(64,64),
                                         batch_size = 32,
                                         class_mode = 'categorical')



Found 21726 images belonging to 10 classes.
Found 5427 images belonging to 10 classes.


In [27]:
def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()

    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=(64,64,3)))

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 2)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    for i in range(hp.Int('n_connections', 1, 2)):
        model.add(Dense(hp.Choice(f'n_nodes',
                                  values=[128, 256, 512])))
        model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [28]:
train_generator.class_indices

{'1. Eczema 1677': 0,
 '10. Warts Molluscum and other Viral Infections - 2103': 1,
 '2. Melanoma 15.75k': 2,
 '3. Atopic Dermatitis - 1.25k': 3,
 '4. Basal Cell Carcinoma (BCC) 3323': 4,
 '5. Melanocytic Nevi (NV) - 7970': 5,
 '6. Benign Keratosis-like Lesions (BKL) 2624': 6,
 '7. Psoriasis pictures Lichen Planus and related diseases - 2k': 7,
 '8. Seborrheic Keratoses and other Benign Tumors - 1.8k': 8,
 '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k': 9}

In [29]:
import keras_tuner as kt
from keras_tuner.engine.hyperparameters import HyperParameters

In [30]:
# pip install keras_tuner

In [31]:
#creating a tuner object
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory='Keras skin disease',
    project_name='Keras tuner skin')

INFO:tensorflow:Reloading Oracle from existing project Keras skin disease\Keras tuner skin\oracle.json
INFO:tensorflow:Reloading Tuner from Keras skin disease\Keras tuner skin\tuner0.json


In [32]:
tuner.search(train_generator,
             epochs=3,
             validation_data=test_generator)

INFO:tensorflow:Oracle triggered exit


In [33]:
tuner.get_best_hyperparameters()[0].values

{'input_units': 64,
 'n_layers': 1,
 'conv_0_units': 96,
 'n_connections': 2,
 'n_nodes': 512,
 'conv_1_units': 32}

In [34]:
model = tuner.get_best_models(num_models=1)[0]

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 activation (Activation)     (None, 62, 62, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 96)        55392     
                                                                 
 activation_1 (Activation)   (None, 29, 29, 96)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 96)       0         
 2D)                                                    

In [36]:
model.fit(train_generator,epochs=15,validation_data=test_generator,initial_epoch=3)

Epoch 4/15
679/679 [==============================] - 409s 603ms/step - loss: 0.9878 - accuracy: 0.6235 - val_loss: 1.0271 - val_accuracy: 0.6173
Epoch 5/15
679/679 [==============================] - 450s 662ms/step - loss: 0.9428 - accuracy: 0.6431 - val_loss: 1.0058 - val_accuracy: 0.6175
Epoch 6/15
679/679 [==============================] - 421s 620ms/step - loss: 0.8944 - accuracy: 0.6626 - val_loss: 1.0052 - val_accuracy: 0.6342
Epoch 7/15
679/679 [==============================] - 418s 616ms/step - loss: 0.8519 - accuracy: 0.6773 - val_loss: 0.9661 - val_accuracy: 0.6422
Epoch 8/15
679/679 [==============================] - 431s 635ms/step - loss: 0.8204 - accuracy: 0.6910 - val_loss: 0.9806 - val_accuracy: 0.6403
Epoch 9/15
679/679 [==============================] - 400s 590ms/step - loss: 0.7937 - accuracy: 0.6998 - val_loss: 0.9395 - val_accuracy: 0.6508
Epoch 10/15
679/679 [==============================] - 359s 529ms/step - loss: 0.7521 - accuracy: 0.7174 - val_loss: 0.9856 

In [41]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(r'C:\Users\mayan\Downloads\New Datasets\skin disease kushal\new\test\10. Warts Molluscum and other Viral Infections - 2103\0_1.jpg', target_size = (64, 64)) # location of image
test_image = image.img_to_array(test_image)    # converting to array
test_image = np.expand_dims(test_image, axis = 0)  # selecting dimension 1 and batch_size
result = model.predict(test_image)
class_names = ['Eczema', 'Warts Molluscum and other Viral Infections', 'Melanoma', 'Atopic Dermatitis',
               'Basal Cell Carcinoma (BCC)', 'Melanocytic Nevi (NV)', 'Benign Keratosis-like Lesions (BKL)', 
               'Psoriasis pictures Lichen Planus and related diseases',
               'Seborrheic Keratoses and other Benign Tumors', 'Tinea Ringworm Candidiasis and other Fungal Infections']
print(class_names[np.argmax(result)])

Warts Molluscum and other Viral Infections


In [43]:
model.save('skin_disease.h5')